In [61]:
import glob, json, os, re

In [66]:
!ls -lh columncache/*/*
!grep -i 'technical doc' columncache/*/description.html

-rw-rw-r--  1 rsargent  staff   3.5M Mar  6  2018 columncache/acs2009_5year_tract2010/description.html
-rw-rw-r--  1 rsargent  staff   5.7M Feb 23  2018 columncache/acs2010_5year_tract2010/description.html
-rw-rw-r--  1 rsargent  staff   3.5M Feb 21  2018 columncache/acs2011_5year_tract2010/description.html
-rw-rw-r--  1 rsargent  staff   2.9M Feb 21  2018 columncache/acs2012_5year_tract2010/description.html
-rw-rw-r--  1 rsargent  staff   3.8M Feb 20  2018 columncache/acs2013_5year_tract2010/description.html
-rw-r--r--  1 rsargent  staff   1.6M Dec 28 06:26 columncache/acs2013_5year_tract2010/description.json
-rw-rw-r--  1 rsargent  staff   4.8M Feb 14  2018 columncache/acs2014_5year_tract2010/description.html
-rw-r--r--  1 rsargent  staff   1.6M Dec 28 06:26 columncache/acs2014_5year_tract2010/description.json
-rw-rw-r--  1 rsargent  staff   6.0M Sep  1  2017 columncache/acs2015_5year_tract2010/description.html
-rw-rw-r--  1 rsargent  staff   6.1M Feb 14  2018 columncache/acs2016_5ye

In [56]:
desc_htmls = glob.glob('columncache/*/description.html')
desc_htmls

['columncache/acs2013_5year_tract2010/description.html',
 'columncache/acs2014_5year_tract2010/description.html',
 'columncache/census1990_block2010/description.html',
 'columncache/acs2015_5year_tract2010/description.html',
 'columncache/census2000_block2010/description.html',
 'columncache/acs2012_5year_tract2010/description.html',
 'columncache/acs2010_5year_tract2010/description.html',
 'columncache/acs2009_5year_tract2010/description.html',
 'columncache/census2010_block2010/description.html',
 'columncache/acs2011_5year_tract2010/description.html',
 'columncache/acs2016_5year_tract2010/description.html']

In [81]:
def desc2json(desc_html):
    desc_json = desc_html.replace('.html', '.json')
    #if os.path.exists(desc_json):
    #    print('%s already exists, skipping' % desc_json)
    #    return
    html = open(desc_html).read()
    lines = html.split("\n")
    print('Read %d lines from %s' % (len(lines), desc_html))

    j = {'tables':[]}

    prev_table_name = None

    for line in lines:
        doc = re.search(r'<a href="(.*)">(.*)</a>', line)
        if doc:
            j['docs'] = [{'url': doc.groups()[0], 'desc': doc.groups()[1]}]
            continue
        match = re.match(r'<tr><td>(.*)</td><td>(.*)</td></tr>', line)
        if not match:
            continue
        (name, desc) = match.groups()
        if not '.' in name:
            name = os.path.basename(os.path.dirname(desc_html)) + '.' + name
        (dataset_name, full_col_name) = name.split('.')
        if '_' in full_col_name:
            (table_name, col_name) = full_col_name.split('_')
        elif len(full_col_name) == 9:
            table_name = full_col_name[:5]
            col_name = full_col_name[5:]
        elif len(full_col_name) == 8:
            table_name = full_col_name[:4]
            col_name = full_col_name[4:]
        elif len(full_col_name) == 7:
            table_name = full_col_name[:4]
            col_name = full_col_name[4:]
        else:
            print(full_col_name)
            raise full_col_name
            
        print(table_name, line)
            
        if not '&mdash;' in desc:
            desc += ' &mdash; Total'
        (table_desc, col_desc) = [x.strip() for x in desc.split('&mdash;', 1)]
        if table_name != prev_table_name:
            j['name'] = dataset_name
            j['tables'].append({'name': table_name, 'description': table_desc, 'columns': []})
            prev_table_name = table_name
        j['tables'][-1]['columns'].append([col_name, col_desc])

    print(len(json.dumps(j)), len(html))

    json.dump(j, open(desc_json, 'w'))
    print(j)
    print('Created %s' % desc_json)

#for desc_html in desc_htmls:
#    desc2json(desc_html)

desc2json("columncache/census1990_block2010/description.html")

Read 986 lines from columncache/census1990_block2010/description.html
H001 <tr><td>H0010001</td><td>Housing Subjects &mdash; Total Housing units</td></tr>
H002 <tr><td>H0020001</td><td>Housing Subjects &mdash; HU Occupancy Status Occupied</td></tr>
H002 <tr><td>H0020002</td><td>Housing Subjects &mdash; HU Occupancy Status Vacant</td></tr>
H003 <tr><td>H0030001</td><td>Housing Subjects &mdash; Owner occupied HUs</td></tr>
H003 <tr><td>H0030002</td><td>Housing Subjects &mdash; Renter occupied HUs</td></tr>
H004 <tr><td>H0040001</td><td>Housing Subjects &mdash; HUs Urban Inside urbanized area</td></tr>
H004 <tr><td>H0040002</td><td>Housing Subjects &mdash; HUs Urban Outside urbanized area</td></tr>
H004 <tr><td>H0040003</td><td>Housing Subjects &mdash; HUs Rural</td></tr>
H004 <tr><td>H0040004</td><td>Housing Subjects &mdash; HUs Nt defined for file</td></tr>
H005 <tr><td>H0050001</td><td>Housing Subjects &mdash; HU Vacancy Status For rent</td></tr>
H005 <tr><td>H0050002</td><td>Housing S

P012 <tr><td>P0120190</td><td>Population Subjects &mdash; Asian/Pacific Isldr Males 5 yrs</td></tr>
P012 <tr><td>P0120191</td><td>Population Subjects &mdash; Asian/Pacific Isldr Males 6 yrs</td></tr>
P012 <tr><td>P0120192</td><td>Population Subjects &mdash; Asian/Pacific Isldr Males 7-9 yrs</td></tr>
P012 <tr><td>P0120193</td><td>Population Subjects &mdash; Asian/Pacific Isldr Males 10-11 yrs</td></tr>
P012 <tr><td>P0120194</td><td>Population Subjects &mdash; Asian/Pacific Isldr Males 12-13 yrs</td></tr>
P012 <tr><td>P0120195</td><td>Population Subjects &mdash; Asian/Pacific Isldr Males 14 yrs</td></tr>
P012 <tr><td>P0120196</td><td>Population Subjects &mdash; Asian/Pacific Isldr Males 15 yrs</td></tr>
P012 <tr><td>P0120197</td><td>Population Subjects &mdash; Asian/Pacific Isldr Males 16 yrs</td></tr>
P012 <tr><td>P0120198</td><td>Population Subjects &mdash; Asian/Pacific Isldr Males 17 yrs</td></tr>
P012 <tr><td>P0120199</td><td>Population Subjects &mdash; Asian/Pacific Isldr Males 18

{'tables': [{'name': 'H001', 'description': 'Housing Subjects', 'columns': [['0001', 'Total Housing units']]}, {'name': 'H002', 'description': 'Housing Subjects', 'columns': [['0001', 'HU Occupancy Status Occupied'], ['0002', 'HU Occupancy Status Vacant']]}, {'name': 'H003', 'description': 'Housing Subjects', 'columns': [['0001', 'Owner occupied HUs'], ['0002', 'Renter occupied HUs']]}, {'name': 'H004', 'description': 'Housing Subjects', 'columns': [['0001', 'HUs Urban Inside urbanized area'], ['0002', 'HUs Urban Outside urbanized area'], ['0003', 'HUs Rural'], ['0004', 'HUs Nt defined for file']]}, {'name': 'H005', 'description': 'Housing Subjects', 'columns': [['0001', 'HU Vacancy Status For rent'], ['0002', 'HU Vacancy Status For sale only'], ['0003', 'HU Vcncy Sts Rnted or sld, not occpied'], ['0004', 'HU Vcncy Sts Season/recreat/occsnl use'], ['0005', 'HU Vcncy Status For migrant workers'], ['0006', 'HU Vacancy Status Other vacant']]}, {'name': 'H006', 'description': 'Housing Subj

In [ ]:
rsync -av --exclude="*.html" columncache hal15:/home/rsargent/uwsgi/dotmaptiles-data/server
